
#  Web Scraping avec Selenium  : Books to Scrape  

##  Introduction  
Le **web scraping** est une technique qui permet d’extraire automatiquement des données à partir de sites web.  

Ici, nous utilisons **Selenium**, une librairie Python permettant d’**automatiser un navigateur web** (comme Chrome, Firefox, etc.) et d’interagir avec les pages (cliquer, remplir des formulaires, naviguer).  




##  Étapes principales avec Selenium  

1️⃣ **Importer les librairies nécessaires**  
2️⃣ **Configurer le driver Chrome** (headless ou normal).  
3️⃣ **Lancer une session navigateur**.  
4️⃣ **Accéder au site cible**.  
5️⃣ **Attendre que les éléments se chargent** (surtout si la page utilise JavaScript).  
6️⃣ **Extraire les informations désirées**.  
7️⃣ **Naviguer sur plusieurs pages** si nécessaire.  
8️⃣ **Sauvegarder les données** dans un fichier CSV.  
9️⃣ **Fermer le navigateur**.  



#  Étape 1 : Web Scraping avec Selenium  

Dans cette première étape, nous allons :  
- Lancer un navigateur automatisé (**Selenium + ChromeDriver**)  
- Explorer le site **Books to Scrape**  
- Récupérer pour chaque livre :  
  - **Title**  
  - **Description**  
  - **Price**  
  - **Availibility**  
  - **Image_URL**  
  - **Rating**  
- Sauvegarder toutes les données dans une structure Python (liste de dictionnaires).  


In [ ]:
%pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\naoui\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Importer des bibliothèques et configurer WebDriver : #

In [1]:

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By   # Pour cibler les éléments
from selenium.webdriver.support.ui import WebDriverWait  # Attente intelligente
from selenium.webdriver.support import expected_conditions as EC  # Conditions d'attente
import os

# Étape 3 : Extraire des données spécifiques de la page #

In [2]:

# === LANCEMENT SELENIUM ===
driver = webdriver.Chrome()  # Lancer Chrome (assurez-vous que chromedriver est accessible)
driver.get("https://books.toscrape.com")  # Ouvrir le site cible

books = []       # Liste pour stocker les données
page_number = 1  # Compteur de pages

# === Boucle de scraping sur toutes les pages ===
while True:
    print(f" Scraping page {page_number}...")
    
    # Attendre que les livres soient chargés
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product_pod"))
    )

    # Attente de 30 secondes pour laisser bien charger (comme demandé)
    time.sleep(30)

    # Récupérer tous les liens des livres sur la page
    book_links = driver.find_elements(By.XPATH, "//h3/a")
    links = [link.get_attribute("href") for link in book_links]

    # Parcourir chaque livre
    for link in links:
        driver.get(link)

        title = driver.find_element(By.TAG_NAME, "h1").text

        try:
            description = driver.find_element(
                By.XPATH, "//div[@id='product_description']/following-sibling::p"
            ).text
        except:
            description = "Pas de description"

        price = driver.find_element(By.CLASS_NAME, "price_color").text
        availability = driver.find_element(By.CLASS_NAME, "availability").text.strip()
        image_url = driver.find_element(By.CSS_SELECTOR, ".item.active img").get_attribute("src")

        rating_class = driver.find_element(By.CLASS_NAME, "star-rating").get_attribute("class").split()[-1]
        rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
        rating = rating_map.get(rating_class, None)

        books.append({
            "Title": title,
            "Description": description,
            "Price": price,
            "Availability": availability,
            "Image_URL": image_url,
            "Rating": rating
        })

    # Retour sur la page pour cliquer sur "Next"
    driver.get(f"https://books.toscrape.com/catalogue/page-{page_number}.html")

    try:
        next_button = driver.find_element(By.CLASS_NAME, "next").find_element(By.TAG_NAME, "a")
        next_page_url = next_button.get_attribute("href")

        # Gestion URL absolue
        if not next_page_url.startswith("http"):
            next_page_url = "https://books.toscrape.com/catalogue/" + next_page_url

        page_number += 1
        driver.get(next_page_url)

    except:
        print(" Fin du scraping - plus de pages.")
        break

# Sauvegarde CSV
df = pd.DataFrame(books)
df.to_csv("books_all_pages.csv", index=False, encoding="utf-8")
print(" Données sauvegardées dans books_all_pages.csv")

driver.quit()


 Scraping page 1...
 Scraping page 2...
 Scraping page 3...
 Scraping page 4...
 Scraping page 5...
 Scraping page 6...
 Scraping page 7...
 Scraping page 8...
 Scraping page 9...
 Scraping page 10...
 Scraping page 11...
 Scraping page 12...
 Scraping page 13...
 Scraping page 14...
 Scraping page 15...
 Scraping page 16...
 Scraping page 17...
 Scraping page 18...
 Scraping page 19...
 Scraping page 20...
 Scraping page 21...
 Scraping page 22...
 Scraping page 23...
 Scraping page 24...
 Scraping page 25...
 Scraping page 26...
 Scraping page 27...
 Scraping page 28...
 Scraping page 29...
 Scraping page 30...
 Scraping page 31...
 Scraping page 32...
 Scraping page 33...
 Scraping page 34...
 Scraping page 35...
 Scraping page 36...
 Scraping page 37...
 Scraping page 38...
 Scraping page 39...
 Scraping page 40...
 Scraping page 41...
 Scraping page 42...
 Scraping page 43...
 Scraping page 44...
 Scraping page 45...
 Scraping page 46...
 Scraping page 47...
 Scraping page 48...
 

In [3]:
df.head(20)

,Title,Description,Price,Availability,Image_URL,Rating
0,A Light in the Attic,It's hard to imagine a world without A Light i...,£51.77,In stock (22 available),https://books.toscrape.com/media/cache/fe/72/f...,3
1,Tipping the Velvet,"""Erotic and absorbing...Written with starling ...",£53.74,In stock (20 available),https://books.toscrape.com/media/cache/08/e9/0...,1
2,Soumission,"Dans une France assez proche de la nôtre, un h...",£50.10,In stock (20 available),https://books.toscrape.com/media/cache/ee/cf/e...,1
3,Sharp Objects,"WICKED above her hipbone, GIRL across her hear...",£47.82,In stock (20 available),https://books.toscrape.com/media/cache/c0/59/c...,4
4,Sapiens: A Brief History of Humankind,From a renowned historian comes a groundbreaki...,£54.23,In stock (20 available),https://books.toscrape.com/media/cache/ce/5f/c...,5
5,The Requiem Red,Patient Twenty-nine.A monster roams the halls ...,£22.65,In stock (19 available),https://books.toscrape.com/media/cache/6b/07/6...,1
6,The Dirty Little Secrets of Getting Your Dream...,Drawing on his extensive experience evaluating...,£33.34,In stock (19 available),https://books.toscrape.com/media/cache/e1/1b/e...,4
7,The Coming Woman: A Novel Based on the Life of...,"""If you have a heart, if you have a soul, Kare...",£17.93,In stock (19 available),https://books.toscrape.com/media/cache/97/36/9...,3
8,The Boys in the Boat: Nine Americans and Their...,For readers of Laura Hillenbrand's Seabiscuit ...,£22.60,In stock (19 available),https://books.toscrape.com/media/cache/d1/2d/d...,4
9,The Black Maria,"Praise for Aracelis Girmay:""[Girmay's] every l...",£52.15,In stock (19 available),https://books.toscrape.com/media/cache/d1/7a/d...,1



#  Étape 2 : Nettoyage & Prétraitement des données  

Maintenant que nous avons collecté les données :  
- Vérifions les valeurs manquantes.  
- Nettoyons les colonnes (ex. prix).  
- Préparons un **DataFrame Pandas** propre pour l’analyse.  


In [4]:
df.shape

(1000, 6)

In [5]:
# Vérifier les valeurs manquantes
print("Valeurs manquantes par colonne :")
print(df.isna().sum())
# Vérifier les valeurs dupliquees
print("Valeurs dupliquees par colonne :")
print(df.duplicated().sum())

Valeurs manquantes par colonne :
Title           0
Description     0
Price           0
Availability    0
Image_URL       0
Rating          0
dtype: int64
Valeurs dupliquees par colonne :
0


In [6]:
df.dtypes

Title           object
Description     object
Price           object
Availability    object
Image_URL       object
Rating           int64
dtype: object

In [7]:
import re

def nettoyer_description(texte):
    if pd.isna(texte):
        return ""
    # Supprimer les caractères spéciaux sauf lettres, chiffres, ponctuation courante
    texte = re.sub(r"[^a-zA-ZÀ-ÿ0-9\s.,!?;:()-]", "", texte)
    # Remplacer espaces multiples par un seul espace
    texte = re.sub(r"\s+", " ", texte)
    # Supprimer espaces au début/fin
    return texte.strip()

df["Description"] = df["Description"].apply(nettoyer_description)


In [8]:
df.Description

0      Its hard to imagine a world without A Light in...
1      Erotic and absorbing...Written with starling p...
2      Dans une France assez proche de la nôtre, un h...
3      WICKED above her hipbone, GIRL across her hear...
4      From a renowned historian comes a groundbreaki...
                             ...                        
995                                   Pas de description
996    High school student Kei Nagai is struck dead i...
997    In Englands Regency era, manners and elegance ...
998    James Patterson, bestselling author of the Ale...
999    Around the World, continent by continent, here...
Name: Description, Length: 1000, dtype: object

# Conversion des colonnes #

In [9]:
df['Price'] = df['Price'].str.replace('£', '').astype(float)


In [10]:
df.dtypes

Title            object
Description      object
Price           float64
Availability     object
Image_URL        object
Rating            int64
dtype: object

In [11]:
# Disponibilité : extraire nombre en stock
def extraire_disponibilite(texte):
    match = re.search(r'\((\d+) available\)', texte)
    if match:
        return int(match.group(1))
    else:
        return 0

df['availability_num'] = df['Availability'].apply(extraire_disponibilite)

In [12]:
df.dtypes

Title                object
Description          object
Price               float64
Availability         object
Image_URL            object
Rating                int64
availability_num      int64
dtype: object

In [13]:
df.head(10)

,Title,Description,Price,Availability,Image_URL,Rating,availability_num
0,A Light in the Attic,Its hard to imagine a world without A Light in...,51.77,In stock (22 available),https://books.toscrape.com/media/cache/fe/72/f...,3,22
1,Tipping the Velvet,Erotic and absorbing...Written with starling p...,53.74,In stock (20 available),https://books.toscrape.com/media/cache/08/e9/0...,1,20
2,Soumission,"Dans une France assez proche de la nôtre, un h...",50.10,In stock (20 available),https://books.toscrape.com/media/cache/ee/cf/e...,1,20
3,Sharp Objects,"WICKED above her hipbone, GIRL across her hear...",47.82,In stock (20 available),https://books.toscrape.com/media/cache/c0/59/c...,4,20
4,Sapiens: A Brief History of Humankind,From a renowned historian comes a groundbreaki...,54.23,In stock (20 available),https://books.toscrape.com/media/cache/ce/5f/c...,5,20
5,The Requiem Red,Patient Twenty-nine.A monster roams the halls ...,22.65,In stock (19 available),https://books.toscrape.com/media/cache/6b/07/6...,1,19
6,The Dirty Little Secrets of Getting Your Dream...,Drawing on his extensive experience evaluating...,33.34,In stock (19 available),https://books.toscrape.com/media/cache/e1/1b/e...,4,19
7,The Coming Woman: A Novel Based on the Life of...,"If you have a heart, if you have a soul, Karen...",17.93,In stock (19 available),https://books.toscrape.com/media/cache/97/36/9...,3,19
8,The Boys in the Boat: Nine Americans and Their...,For readers of Laura Hillenbrands Seabiscuit a...,22.60,In stock (19 available),https://books.toscrape.com/media/cache/d1/2d/d...,4,19
9,The Black Maria,Praise for Aracelis Girmay:Girmays every losss...,52.15,In stock (19 available),https://books.toscrape.com/media/cache/d1/7a/d...,1,19


In [14]:
df.describe()

,Price,Rating,availability_num
count,1000.00000,1000.000000,1000.000000
mean,35.07035,2.923000,8.585000
std,14.44669,1.434967,5.654622
min,10.00000,1.000000,1.000000
25%,22.10750,2.000000,3.000000
50%,35.98000,3.000000,7.000000
75%,47.45750,4.000000,14.000000
max,59.99000,5.000000,22.000000


L’analyse descriptive des données montre que **les prix** des produits varient entre 10 et 59,99, avec un prix moyen d’environ 35. Les évaluations **(ratings)** sont comprises entre 1 et 5, avec une moyenne proche de 2,9, indiquant une tendance générale vers des notes modérées. Quant à la disponibilité des produits, elle oscille entre 1 et 22 unités, avec une moyenne de 8,6, suggérant que la plupart des articles sont disponibles en petite à moyenne quantité. Les quartiles confirment cette distribution : 25 % des produits coûtent moins de 22,10, 50 % ont un rating inférieur ou égal à 3 et la moitié des articles ont jusqu’à 7 unités en stock.

In [39]:
df.to_csv("livres_bruts.csv", index=False, encoding="utf-8")
